In [2]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd

In [3]:
pothole = pd.read_csv("https://data.cityofchicago.org/resource/7as2-ds3y.csv?$limit=1000000")

In [4]:
patched = pd.read_csv("https://data.cityofchicago.org/resource/wqdh-9gek.csv?$limit=1000000")

In [5]:
pothole.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560478 entries, 0 to 560477
Data columns (total 19 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   creation_date                       560478 non-null  object 
 1   status                              560478 non-null  object 
 2   completion_date                     553130 non-null  object 
 3   service_request_number              560478 non-null  object 
 4   type_of_service_request             560478 non-null  object 
 5   current_activity                    393609 non-null  object 
 6   most_recent_action                  391992 non-null  object 
 7   number_of_potholes_filled_on_block  399143 non-null  float64
 8   street_address                      560476 non-null  object 
 9   zip                                 558160 non-null  float64
 10  x_coordinate                        560004 non-null  float64
 11  y_coordinate              

In [6]:
pothole.shape

(560478, 19)

In [7]:
pothole.columns

Index(['creation_date', 'status', 'completion_date', 'service_request_number',
       'type_of_service_request', 'current_activity', 'most_recent_action',
       'number_of_potholes_filled_on_block', 'street_address', 'zip',
       'x_coordinate', 'y_coordinate', 'ward', 'police_district',
       'community_area', 'ssa', 'latitude', 'longitude', 'location'],
      dtype='object')

In [8]:
patched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179958 entries, 0 to 179957
Data columns (total 7 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   address                             179958 non-null  object 
 1   request_date                        179958 non-null  object 
 2   completion_date                     179958 non-null  object 
 3   number_of_potholes_filled_on_block  179958 non-null  int64  
 4   latitude                            179489 non-null  float64
 5   longitude                           179489 non-null  float64
 6   location                            179489 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 9.6+ MB


In [9]:
patched.shape

(179958, 7)

In [10]:
patched.columns

Index(['address', 'request_date', 'completion_date',
       'number_of_potholes_filled_on_block', 'latitude', 'longitude',
       'location'],
      dtype='object')

In [11]:
pothole.sample(1)

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
496272,2011-07-24T00:00:00.000,Completed,2011-08-03T00:00:00.000,11-03447773,Pot Hole in Street,Dispatch Crew,Pothole Patched,3.0,8000 S LANGLEY AVE,60619.0,1.182283e+06,1.852114e+06,6.0,6.0,44.0,NaN,41.749323,-87.607755,"(41.749323113054, -87.607755105857)"


In [12]:
potholes_completed = pothole[pothole['status'] == 'Completed']

In [13]:
potholes_completed

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
42,2018-12-17T00:00:00.000,Completed,2018-12-18T00:00:00.000,18-03378095,Pothole in Street,Final Outcome,Pothole Patched,7.0,600 W LAKE ST,60661.0,1.172289e+06,1.901697e+06,42.0,12.0,28.0,NaN,41.885815,-87.642847,"(41.885814724037, -87.6428471339)"
58,2018-12-17T00:00:00.000,Completed,2018-12-18T00:00:00.000,18-03378491,Pothole in Street,Final Outcome,Completed Upon Arrival,0.0,719 N DRAKE AVE,60624.0,1.152585e+06,1.904606e+06,27.0,11.0,23.0,63.0,41.894335,-87.714912,"(41.894335117217, -87.714912418158)"
236,2018-12-15T00:00:00.000,Completed,2018-12-17T00:00:00.000,18-03366015,Pothole in Street,Final Outcome,Pothole Patched,4.0,230 N OGDEN AVE,60607.0,1.166961e+06,1.901783e+06,27.0,12.0,28.0,NaN,41.885849,-87.662604,"(41.885849434411, -87.662603973878)"
263,2018-12-15T00:00:00.000,Completed,2018-12-31T00:00:00.000,18-03365265,Pothole in Street,Final Outcome,Pothole Patched,17.0,8200 S SPRINGFIELD AVE,60652.0,1.151820e+06,1.849922e+06,18.0,8.0,70.0,NaN,41.744008,-87.719444,"(41.744007628888, -87.719443672392)"
289,2018-12-14T00:00:00.000,Completed,2018-12-14T00:00:00.000,18-03360709,Pothole in Street,Final Outcome,Pothole Patched,10.0,2844 W GRACE ST,60618.0,1.156639e+06,1.925124e+06,33.0,17.0,16.0,NaN,41.950429,-87.699970,"(41.950429023486, -87.6999704208)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560473,2010-03-11T00:00:00.000,Completed,2011-03-28T00:00:00.000,10-00317212,Pot Hole in Street,Dispatch Crew,Pothole Patched,5.0,7201 W BELMONT AVE,60634.0,NaN,NaN,36.0,25.0,17.0,NaN,41.937886,-87.806806,"(41.937885966284, -87.8068055064)"
560474,2010-02-26T00:00:00.000,Completed,2011-10-04T00:00:00.000,10-00254019,Pot Hole in Street,NaN,NaN,NaN,1440 S ROCKWELL ST,60608.0,1.159269e+06,1.892879e+06,28.0,10.0,29.0,NaN,41.861757,-87.690973,"(41.861756644625, -87.690972506101)"
560475,2009-08-27T00:00:00.000,Completed,2011-01-25T00:00:00.000,09-01501980,Pot Hole in Street,Site Survey,No Problem Found,80.0,5518 N WAYNE AVE,60640.0,NaN,NaN,48.0,20.0,77.0,NaN,41.982340,-87.663869,"(41.98234020574, -87.663868977471)"
560476,2009-06-17T00:00:00.000,Completed,2011-06-07T00:00:00.000,09-01026546,Pot Hole in Street,Dispatch Crew,Pothole Patched,NaN,3845 W ADDISON ST,60618.0,1.150061e+06,1.923655e+06,30.0,17.0,21.0,NaN,41.946303,-87.724348,"(41.946303283819, -87.724347984027)"


In [240]:
potholes_no_problem = pothole['most_recent_action'] == 'No Problem Found'

In [241]:
potholes_no_problem

0         False
1         False
2         False
3         False
4         False
          ...  
560473    False
560474    False
560475     True
560476    False
560477    False
Name: most_recent_action, Length: 560478, dtype: bool

In [215]:
sample_pothole = pothole.sample(5)

In [216]:
sample_pothole

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
407739,2013-04-08T00:00:00.000,Completed,2013-04-15T00:00:00.000,13-00394338,Pot Hole in Street,Final Outcome,Pothole Patched,30.0,2000 N KEDVALE AVE,60639.0,1.148396e+06,1.912965e+06,30.0,25.0,20.0,NaN,41.917203,-87.730376,"(41.91720314213, -87.730376009385)"
5150,2018-10-22T00:00:00.000,Open,NaN,18-02972453,Pothole in Street,NaN,NaN,NaN,1948 W 79TH ST,60620.0,1.164639e+06,1.852272e+06,17.0,6.0,71.0,NaN,41.750346,-87.672712,"(41.750345612106, -87.672712203174)"
193222,2015-07-06T00:00:00.000,Completed,2015-09-15T00:00:00.000,15-03231652,Pothole in Street,Final Outcome,Pothole Patched,21.0,3814 N DAMEN AVE,60618.0,1.162334e+06,1.925386e+06,47.0,19.0,5.0,NaN,41.950965,-87.678814,"(41.950964934439, -87.678813846176)"
274966,2014-05-27T00:00:00.000,Completed,2014-10-04T00:00:00.000,14-00790161,Pothole in Street,Final Outcome,No Potholes Found,0.0,2127 W HOOD AVE,60659.0,1.160905e+06,1.940718e+06,40.0,24.0,2.0,NaN,41.992913,-87.683511,"(41.9929131322, -87.683511104559)"
5048,2018-10-23T00:00:00.000,Open - Dup,NaN,18-02987522,Pothole in Street,NaN,NaN,NaN,5927 N RIDGE AVE,60660.0,1.164693e+06,1.939448e+06,48.0,20.0,77.0,NaN,41.989674,-87.669704,"(41.989674336381, -87.669703506677)"


In [192]:
for column in sample_pothole.iterrows():
  if column[1]['status'] == 'Completed':
    print('its completed!')
  elif column[1]['status'] == 'Completed - Dup':
    print('its completed!')
  else:
    print('it hasnt been completed')

its completed!
its completed!
its completed!
its completed!
its completed!


In [229]:
open_requests = pothole['status'].isin(['Open', 'Open - Dup'])

In [231]:
open_requests.value_counts()

status
False    553130
True       7348
Name: count, dtype: int64

In [232]:
potholes_patched = pothole['most_recent_action'] == 'Pothole Patched'

In [234]:
potholes_patched.value_counts()

most_recent_action
True     317646
False    242832
Name: count, dtype: int64

In [239]:
potholes_no_problem.value_counts()

most_recent_action
False    560178
True        300
Name: count, dtype: int64